# CREDIT CARD FRAUD  
Group: Frank Abagnale assistants \
Aurora Ciandri - 2024578 \
Ellada Aslanova - 20094473 \
Shireesha Pothuganti - 2008492

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels.api as sm

#Packages for the design of features part
from xverse.transformer import MonotonicBinning
from xverse.transformer import WOE

#Packages for the training and validation part
import imblearn
import sklearn
import collections
import sklearn.metrics as metrics

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
#import data
url = '/Users/aurora/Desktop/UniPD/FIS/project-fis/creditcard.csv'
data = pd.read_csv(url)
data.sample(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
44356,41914.0,-1.162351,-1.866244,0.519368,0.026778,-3.052502,1.313570,0.629101,0.601345,-0.588191,...,0.798133,1.411664,1.357064,0.008363,-0.860512,-0.093473,-0.001441,0.193253,583.32,0
203143,134671.0,2.279457,-0.552937,-2.426203,-0.808606,-0.042905,-1.997751,0.410793,-0.637362,-0.815361,...,0.464995,1.340944,-0.216020,0.106957,0.706699,0.301992,-0.107306,-0.094886,20.00,0
272865,165302.0,2.196721,-0.989503,-0.728671,-0.914782,-0.895526,-0.468733,-0.971291,-0.130428,0.154243,...,0.284287,0.908772,0.041466,-0.611061,-0.054546,-0.048457,0.009470,-0.053094,19.95,0
133340,80343.0,1.232770,-0.114556,-0.051480,0.339765,0.275117,0.694202,-0.166082,0.119077,0.296357,...,-0.332225,-0.869339,-0.187382,-1.351867,0.544503,0.264264,-0.024323,-0.003015,42.47,0
256603,157771.0,0.124285,0.621314,0.350324,-0.998058,0.520489,-0.353372,0.469196,-0.891391,-0.307399,...,0.473240,-1.125809,-0.182608,-0.399056,0.915010,0.232855,0.057637,0.179180,1.98,0
2376,1929.0,1.107855,0.628333,-0.363276,1.129980,0.577495,0.039454,0.100199,0.140357,-0.464608,...,0.048095,0.279351,-0.084118,-0.401166,0.537160,-0.287881,0.069189,0.042035,1.00,0
183489,125826.0,-0.725460,0.028038,-1.379000,-0.122900,-0.350420,-0.006784,2.475858,-0.433046,-0.294623,...,0.289775,1.066087,0.224496,-0.539521,-0.170652,-0.131967,0.394447,0.154049,442.86,0
20671,31198.0,-1.331040,0.519220,1.333915,-0.721706,0.442685,1.844601,-0.410840,1.277316,-0.173956,...,0.353142,1.120865,-0.076059,-1.296679,-0.298831,0.578759,0.260083,0.066003,25.65,0
74107,55431.0,1.117473,-0.338531,0.252422,0.748974,-0.512306,-0.241361,-0.132024,0.021216,0.815393,...,-0.316315,-1.000804,-0.063683,-0.351637,0.349879,0.222137,-0.041987,0.020355,86.95,0
40391,40218.0,1.192995,0.532702,-0.414619,1.320924,0.093468,-0.821472,0.099055,0.006376,0.207197,...,-0.063027,-0.170890,-0.070136,-0.113890,0.550846,-0.291011,0.038699,0.049808,1.00,0


## EDA (Explorative Data Analysis)
We print some useful information about our data first.

In [3]:
nfraud = data ['Class']==0
fraud = data ['Class']==1

In [4]:
print("Number of observations = {}".format(data.shape[0]))
print( 'Number of observations with no fraud = {}'.format(data[nfraud].shape[0]))
print( 'Number of observations with fraud = {}'.format(data[fraud].shape[0]))

Number of observations = 284807
Number of observations with no fraud = 284315
Number of observations with fraud = 492


In [5]:
print("Number of fields = {}".format(data.shape[1]))
print("Column names = [{}]".format(", ".join([c for c in data.columns])))
print("The data types of each column are:\n{}".format(data.dtypes))

Number of fields = 31
Column names = [Time, V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V20, V21, V22, V23, V24, V25, V26, V27, V28, Amount, Class]
The data types of each column are:
Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object


We then proceed with some summary statistics on the variables in the dataset, firstly for all the observations and then separately for the fraud and non-fraud observations.

In [6]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [7]:
data[nfraud].describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,...,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.0
mean,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022,0.0
std,47484.015786,1.929814,1.636146,1.459429,1.399333,1.356952,1.329913,1.178812,1.161283,1.089372,...,0.716743,0.723668,0.621541,0.605776,0.520673,0.482241,0.399847,0.329570,250.105092,0.0
min,0.000000,-56.407510,-72.715728,-48.325589,-5.683171,-113.743307,-26.160506,-31.764946,-73.216718,-6.290730,...,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-15.430084,0.000000,0.0
25%,54230.000000,-0.917544,-0.599473,-0.884541,-0.850077,-0.689398,-0.766847,-0.551442,-0.208633,-0.640412,...,-0.228509,-0.542403,-0.161702,-0.354425,-0.317145,-0.327074,-0.070852,-0.052950,5.650000,0.0
50%,84711.000000,0.020023,0.064070,0.182158,-0.022405,-0.053457,-0.273123,0.041138,0.022041,-0.049964,...,-0.029821,0.006736,-0.011147,0.041082,0.016417,-0.052227,0.001230,0.011199,22.000000,0.0
75%,139333.000000,1.316218,0.800446,1.028372,0.737624,0.612181,0.399619,0.571019,0.326200,0.598230,...,0.185626,0.528407,0.147522,0.439869,0.350594,0.240671,0.090573,0.077962,77.050000,0.0
max,172792.000000,2.454930,18.902453,9.382558,16.875344,34.801666,73.301626,120.589494,18.709255,15.594995,...,22.614889,10.503090,22.528412,4.584549,7.519589,3.517346,31.612198,33.847808,25691.160000,0.0


In [8]:
data[fraud].describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,...,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.0
mean,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321,1.0
std,47835.365138,6.783687,4.291216,7.110937,2.873318,5.372468,1.858124,7.206773,6.797831,2.500896,...,3.869304,1.494602,1.579642,0.515577,0.797205,0.471679,1.376766,0.547291,256.683288,0.0
min,406.000000,-30.552380,-8.402154,-31.103685,-1.313275,-22.105532,-6.406267,-43.557242,-41.044261,-13.434066,...,-22.797604,-8.887017,-19.254328,-2.028024,-4.781606,-1.152671,-7.263482,-1.869290,0.000000,1.0
25%,41241.500000,-6.036063,1.188226,-8.643489,2.373050,-4.792835,-2.501511,-7.965295,-0.195336,-3.872383,...,0.041787,-0.533764,-0.342175,-0.436809,-0.314348,-0.259416,-0.020025,-0.108868,1.000000,1.0
50%,75568.500000,-2.342497,2.717869,-5.075257,4.177147,-1.522962,-1.424616,-3.034402,0.621508,-2.208768,...,0.592146,0.048434,-0.073135,-0.060795,0.088371,0.004321,0.394926,0.146344,9.250000,1.0
75%,128483.000000,-0.419200,4.971257,-2.276185,6.348729,0.214562,-0.413216,-0.945954,1.764879,-0.787850,...,1.244611,0.617474,0.308378,0.285328,0.456515,0.396733,0.826029,0.381152,105.890000,1.0
max,170348.000000,2.132386,22.057729,2.250210,12.114672,11.095089,6.474115,5.802537,20.007208,3.353525,...,27.202839,8.361985,5.466230,1.091435,2.208209,2.745261,3.052358,1.779364,2125.870000,1.0


We also created 2 graphs to study the dispersion of the data we're analyzing and the correlation among the variables. As they underwent a PCA (Principal Component Analysis), we expect them to present no correlation.

In [9]:
data2 = data.copy()
del(data2['Time'])
del(data2['Amount'])
sns.boxplot(data = data2)

<AxesSubplot:>

In [10]:
sns.boxplot(data = data2[fraud])

<AxesSubplot:>

From this graph, we can see that there are quite a lot of outliers and some of the variables show a great dispersion.

In [11]:
# Create a Figure object
fig = plt.figure()

# Add a 2x2 subplot and selecting the 1st one (numbering starts from 1)
ax1 = fig.add_subplot(2, 2, 1)

# heatmap of correlation
sns.heatmap(data.corr())

ax2 = fig.add_subplot(2, 2, 2)
sns.heatmap((data.corr()>.7) | (data.corr()<-.7))

<AxesSubplot:>

As expected, there is no significant correlation among the variables we're studying.

In [12]:
fig = plt.figure()

ax1 = fig.add_subplot(2, 2, 1)
sns.heatmap((data[fraud].corr()>.7) | (data[fraud].corr()<-.7))

ax2 = fig.add_subplot(2, 2, 3)
sns.heatmap((data[nfraud].corr()>.7) | (data[nfraud].corr()<-.7))

<AxesSubplot:>

In [13]:
np.random.seed(100)
np.random.randint( 30, size = 3) #8, 24, 3

fig = plt.figure()

ax1 = fig.add_subplot(2, 2, 1)
p1 = data['Amount'][nfraud].plot(label = 'not fraud')
data['Amount'][fraud].plot(label = 'fraud')
p1.legend(loc="upper right", title="Legend", frameon=False)

ax2 = fig.add_subplot(2, 2, 2)
p2 = data['V8'][nfraud].plot(label = 'not fraud')
data['V8'][fraud].plot(label = 'fraud')
#p2.legend(loc="lower right", title="Legend", frameon=False)

ax3 = fig.add_subplot(2, 2, 3)
p2 = data['V24'][nfraud].plot(label = 'not fraud')
data['V24'][fraud].plot(label = 'fraud')
#p2.legend(loc="lower right", title="Legend", frameon=False)

ax4 = fig.add_subplot(2, 2, 4)
p2 = data['V3'][nfraud].plot(label = 'not fraud')
data['V3'][fraud].plot(label = 'fraud')
#p2.legend(loc="lower right", title="Legend", frameon=False)

<AxesSubplot:>

## Data Cleaning
We verified the presence of missing values, duplicated rows and outliers. For the first two cases, we then proceeded to delete them, while for the last one we kept them. \
This last part was done through their transformation into either the higher or the lower extreme of the interval computed through the IQR (InterQuartile Range). That was done in order to keep the data as close at the original as possible, so that we can compute them without worrying about possible biases introduced by deleting them.

In [14]:
#missing data
print("Q: Is there at least a missing value in our DataFrame? A: {}".
      format(data.isnull().any().any()))

#duplicates
print("Q: Is there at least one duplicated row? A: {}"
      .format(data.duplicated().any())) 
data1 = data.drop_duplicates()
print("Q: Is there at least one duplicated row? A: {}"
      .format(data1.duplicated().any()))
print()
print("Total number of rows after removing duplicated rows = {}"
      .format(data1.shape[0]))

Q: Is there at least a missing value in our DataFrame? A: False
Q: Is there at least one duplicated row? A: True
Q: Is there at least one duplicated row? A: False

Total number of rows after removing duplicated rows = 283726


In [15]:
l = []

def out_cleaner(dati):
    dati2 = dati.copy()
    obs = 0
    Q1 = dati.quantile(0.25) # 1° quartile
    Q3 = dati.quantile(0.75) # 3° quartile
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR      # lower whiskey
    upper = Q3 + 1.5 * IQR      # upper whiskey
    
    for col in range(dati2.shape[1]-1):
        var = dati2.columns[col]
        up = upper[col]
        low = lower[col]
        
        if np.max(dati2[var]) <= up and np.min(dati2[var]) >= low :
            continue
        else:
            dati2[var].apply(change, args = (up, low))
            
        outliers = len(l)
    print ( 'There were {} outliers in our data.'.format (outliers))
    return dati2
        
def change (x, up, low):
    if x > up:
        l.append(x)
        x = up
    elif x < low:
        l.append(x)
        x = low      
        
clean_data = out_cleaner(data1)

There were 367659 outliers in our data.


In [16]:
print( 'We\'ve got {} fraud observations in the clean data.'.format(clean_data[fraud].shape[0]))
print( 'We\'ve got {} non-fraud observations in the clean data.'.format(clean_data[nfraud].shape[0]))
print( 'Therefore, we\'ve got that {}% of the observations are frauds.'.format(clean_data[fraud].shape[0]/clean_data.shape[0]*100))

We've got 473 fraud observations in the clean data.
We've got 283253 non-fraud observations in the clean data.
Therefore, we've got that 0.1667101358352777% of the observations are frauds.


At the end of this process : 
- we found and deleted 1081 duplicates from our data ;
- we found and dealt with 367659 outliers as specified before.

## Features selection

For this part of the project, we compute the Information Value (IV), which helps rank the variables according to their importance for the prediction process in order to select the most important features.
\
In order to do that, we start by splitting the dataset according to the "Class" variable (so "fraud" and "non fraud" observations) and we transform it into a numpy array.

In [17]:
def prep_dataset():
    from xverse.feature_subset import SplitXY

    clf = SplitXY(['Class']) #Split the dataset into X and y
    X, y = clf.fit_transform(clean_data.drop('Time', axis = 1)) #returns features dataset (X) and target(Y) as a numpy array
    
    return X, y

X, y = prep_dataset()

As the IV is based on the Weight Of Evidence (WOE)* , we proceed to bin the variables into buckets according to the quantity of information they  deliver to a potential classification model thorugh Monotonic Binning*. \
*The WOE is a indicator of the predictive power of an independent variable on the dependent one usually used in financial data analysis. \
*Monotonic Binning is a data preparation technique widely used in scorecard development, that tries to convert numerical variables into categorical ones by creating bins which have a monotonic relationship with the target.

In [18]:
clf = MonotonicBinning()
clf.fit(X, y)
clf.bins

{'Amount': array([0.000000e+00, 9.990000e+00, 5.000000e+01, 2.569116e+04]),
 'V1': array([-56.40750963,  -0.63719512,   1.17798035,   2.45492999]),
 'V10': array([-24.58826244,  -0.36533763,   0.18965754,  23.74513612]),
 'V11': array([-4.79747346, -0.51562296,  0.47328399, 12.01891318]),
 'V12': array([-18.68371463,  -0.19489697,   0.44908537,   7.84839208]),
 'V13': array([-5.79188121, -0.40604068,  0.41025353,  7.12688296]),
 'V14': array([-19.21432549,  -0.24607389,   0.33053773,  10.52676605]),
 'V15': array([-4.49894468, -0.35426597,  0.43117117,  8.8777416 ]),
 'V16': array([-14.12985452,  -0.26153312,   0.36473537,  17.31511152]),
 'V17': array([-25.16279937,  -0.34404216,   0.22910506,   9.25352625]),
 'V18': array([-9.49874592, -0.3147951 ,  0.31412414,  5.04106919]),
 'V19': array([-7.21352743, -0.2767006 ,  0.28055845,  5.59197143]),
 'V2': array([-72.71572756,  -0.33783808,   0.51722944,  22.05772899]),
 'V20': array([-54.49772049,  -0.15853903,   0.0552205 ,  39.42090425]

In [19]:
out_X = clf.transform(X) #Convert the array in a dataset
out_X.head(5)

,Amount,V1,V10,V11,V12,V13,V14,V15,V16,V17,...,V26,V27,V28,V3,V4,V5,V6,V7,V8,V9
0,"(50.0, 25691.16]","(-56.409, -0.637]","(-0.365, 0.19]","(-4.798, -0.516]","(-18.685000000000002, -0.195]","(-5.793, -0.406]","(-19.215, -0.246]","(0.431, 8.878]","(-14.131, -0.262]","(-0.344, 0.229]",...,"(-0.239, 0.141]","(0.0494, 31.612]","(-0.032, 0.0411]","(0.724, 9.383]","(0.459, 16.875]","(-0.454, 0.365]","(0.124, 73.302]","(-0.332, 0.359]","(-0.127, 0.196]","(0.356, 15.595]"
1,"(-0.001, 9.99]","(1.178, 2.455]","(-0.365, 0.19]","(0.473, 12.019]","(0.449, 7.848]","(0.41, 7.127]","(-0.246, 0.331]","(0.431, 8.878]","(0.365, 17.315]","(-0.344, 0.229]",...,"(-0.239, 0.141]","(-0.0421, 0.0494]","(-0.032, 0.0411]","(-0.466, 0.724]","(-0.603, 0.459]","(-0.454, 0.365]","(-0.603, 0.124]","(-0.332, 0.359]","(-0.127, 0.196]","(-0.414, 0.356]"
2,"(50.0, 25691.16]","(-56.409, -0.637]","(0.19, 23.745]","(0.473, 12.019]","(-0.195, 0.449]","(0.41, 7.127]","(-0.246, 0.331]","(0.431, 8.878]","(-14.131, -0.262]","(0.229, 9.254]",...,"(-0.239, 0.141]","(-22.567, -0.0421]","(-15.431, -0.032]","(0.724, 9.383]","(-0.603, 0.459]","(-113.744, -0.454]","(0.124, 73.302]","(0.359, 120.589]","(0.196, 20.007]","(-13.434999999999999, -0.414]"
3,"(50.0, 25691.16]","(-56.409, -0.637]","(-0.365, 0.19]","(-0.516, 0.473]","(-0.195, 0.449]","(0.41, 7.127]","(-19.215, -0.246]","(-4.5, -0.354]","(-14.131, -0.262]","(-25.164, -0.344]",...,"(-0.239, 0.141]","(0.0494, 31.612]","(0.0411, 33.848]","(0.724, 9.383]","(-5.684, -0.603]","(-0.454, 0.365]","(0.124, 73.302]","(-0.332, 0.359]","(0.196, 20.007]","(-13.434999999999999, -0.414]"
4,"(50.0, 25691.16]","(-56.409, -0.637]","(0.19, 23.745]","(-4.798, -0.516]","(0.449, 7.848]","(0.41, 7.127]","(-19.215, -0.246]","(-0.354, 0.431]","(-14.131, -0.262]","(-0.344, 0.229]",...,"(0.141, 3.517]","(0.0494, 31.612]","(0.0411, 33.848]","(0.724, 9.383]","(-0.603, 0.459]","(-0.454, 0.365]","(-0.603, 0.124]","(0.359, 120.589]","(-73.218, -0.127]","(0.356, 15.595]"


In [20]:
clf = WOE()
clf.fit(X, y)

WOE(mono_custom_binning={'Amount': array([0.000000e+00, 9.990000e+00, 5.000000e+01, 2.569116e+04]),
                         'V1': array([-56.40750963,  -0.63719512,   1.17798035,   2.45492999]),
                         'V10': array([-24.58826244,  -0.36533763,   0.18965754,  23.74513612]),
                         'V11': array([-4.79747346, -0.51562296,  0.47328399, 12.01891318]),
                         'V12': array([-18.68371463,  -0.19489697,   0.44908537,   7.84839208]),
                         'V13': a...
              'V8': {Interval(-73.218, -0.127, closed='right'): -0.24867289819766109,
                     Interval(-0.127, 0.196, closed='right'): -1.0362040790456226,
                     Interval(0.196, 20.007, closed='right'): 0.6245451853340575},
              'V9': {Interval(-13.434999999999999, -0.414, closed='right'): 0.8951638013816031,
                     Interval(-0.414, 0.356, closed='right'): -1.1698095281117673,
                     Interval(0.356, 15.595, clos

In [21]:
clf.woe_df.head(10)

,Variable_Name,Category,Count,Event,Non_Event,Event_Rate,Non_Event_Rate,Event_Distribution,Non_Event_Distribution,WOE,Information_Value
0,Amount,"(-0.001, 9.99]",96873,238,96635,0.002457,0.997543,0.503171,0.341161,0.388575,0.291657
1,Amount,"(9.99, 50.0]",93275,55,93220,0.000590,0.999410,0.116279,0.329105,-1.040384,0.291657
2,Amount,"(50.0, 25691.16]",93578,180,93398,0.001924,0.998076,0.380550,0.329733,0.143332,0.291657
3,V1,"(-56.409, -0.637]",94576,340,94236,0.003595,0.996405,0.718816,0.332692,0.770388,0.806540
4,V1,"(-0.637, 1.178]",94575,104,94471,0.001100,0.998900,0.219873,0.333522,-0.416657,0.806540
5,V1,"(1.178, 2.455]",94575,29,94546,0.000307,0.999693,0.061311,0.333786,-1.694546,0.806540
6,V10,"(-24.589000000000002, -0.365]",94576,422,94154,0.004462,0.995538,0.892178,0.332402,0.987319,1.608221
7,V10,"(-0.365, 0.19]",94579,18,94561,0.000190,0.999810,0.038055,0.333839,-2.171628,1.608221
8,V10,"(0.19, 23.745]",94571,33,94538,0.000349,0.999651,0.069767,0.333758,-1.565249,1.608221
9,V11,"(-4.798, -0.516]",94576,19,94557,0.000201,0.999799,0.040169,0.333825,-2.117519,1.680390


In [22]:
clf.iv_df

,Variable_Name,Information_Value
23,V4,2.430793
6,V14,2.151082
4,V12,2.005756
22,V3,1.704309
3,V11,1.680390
2,V10,1.608221
12,V2,1.248325
8,V16,1.233279
9,V17,1.074592
28,V9,1.051508


From this last dataframe, we can see that almost all variables (but V15, V22 and V25) seem to have predictive power, even though weak for V24, V13 and V26. We can also notice that some of the variables have informative values that reach very high values: according to Shiddiqi, values higher than 0.5 are suspicious as they seem too good to be true. It might in fact indicate that such variables are related to the dependent one. \
In our case, it seems that the variables are incorreleted with each other (according to both the theory of the PCA and the heat graph presented previously), but we still need to check it through the Variance Inflation Factor, which measures how much of the variance of the OLS coefficients is increased because of collinearity. \
\
Therefore, we proceed to the computation of the VIF on the variables with predictive power.

In [23]:
ivars = clf.iv_df[clf.iv_df['Information_Value']>.02]
ivars = ivars['Variable_Name']

In [24]:
def vif_cal(dati, dependent_col):
    vif_df = pd.DataFrame( columns = ['Var', 'Vif'])
    x_vars = dati.drop([dependent_col], axis=1)
    xvar_names = x_vars.columns
    for i in range( 0 , xvar_names.shape[0] ) :
        y = x_vars[xvar_names[i]] 
        x = x_vars[xvar_names.drop(xvar_names[i])]
        rsq = sm.OLS(y,x).fit().rsquared  
        vif = round(1/(1-rsq),2)
        vif_df.loc[i] = [xvar_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis=0, ascending=False, inplace=False)

In [25]:
names = list(ivars.values)
names.append('Class')
print(names)

dataa = clean_data.copy()
dataa = dataa[names]

['V4', 'V14', 'V12', 'V3', 'V11', 'V10', 'V2', 'V16', 'V17', 'V9', 'V7', 'V1', 'V21', 'V6', 'V27', 'V28', 'V18', 'V5', 'V8', 'V19', 'V20', 'Amount', 'V23', 'V24', 'V13', 'V26', 'Class']


In [26]:
vif_cal(dataa, 'Class')

,Var,Vif
21,Amount,5.30
6,V2,2.33
10,V7,1.73
17,V5,1.69
20,V20,1.54
11,V1,1.24
13,V6,1.22
3,V3,1.21
22,V23,1.06
18,V8,1.05


Our results show values small enough to conclude the lack of correlation among the variables, as the heatmap showed. The only variable that show a very strong correlation to the dependent variable is the 'Amount', which, as a consequence, must be removed.

## Training and validation

After cleaning the data and computing the predictive relevance of each variable, we then proceed to create the train set and the test set. \
As we're dealing with very imbalanced data, we oversampled the minority class (in our case fraud observations) through the Synthetic Minority Oversampling Technique (SMOTE) in order to improve the bias that favor the majority class (non-fraud). \
We took 80% of the data for the train set and the 20% of it for the test set.

In [27]:
X = dataa.drop(["Class",'Amount'], axis = 1)
y = dataa['Class']
# transform the dataset
oversample = imblearn.over_sampling.SMOTE()
X, y = oversample.fit_resample(X, y)

# summarize the new class distribution
counter = collections.Counter(y)
print(counter)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, 
                                                    stratify = y, shuffle=True, random_state = 123)

Counter({0: 283253, 1: 283253})


### Procedure
For the classification procedure we used 2 different models in order to do the comparison, i.e. Logistic regression and XGboost. \
\
On each of them we used the following tools:
- Accuracy score : it is used to evaluate classification models. It is calculated simply by dividing the number of correct predictions made by the model by the total number of predictions made by the model.
- The F1 score : it can be defined as the harmonic mean of the model’s precision and recall. It is calculated by dividing the product of the model’s precision and recall by the value obtained on adding the model’s precision and recall and finally multiplying the result with 2.

Furthermore, we computed the confusion matrix* for both of them in order to measure and compare their accuracy: this part of the code prints out the number of false positives it detected and compares it with the actual values in order to calculate accuracy score and precision. 

*In the confusion matrix the first row is for transactions whose actual fraud value in the test set is 0.The second row tells the number of transactions whose fraud value is 1.

### Logistic Regression

In [28]:
model = LogisticRegression()
model.fit(X_train, y_train)
print('For train set\n')
print(classification_report(model.predict(X_train), y_train))
print('Confusion matrix (train set):\n')
print(confusion_matrix(y_train, model.predict(X_train)))

print()
print()

print('For test set\n')
print(classification_report(model.predict(X_test), y_test))
print('Confusion matrix (test set):\n')
print(confusion_matrix(y_test, model.predict(X_test)))

For train set

              precision    recall  f1-score   support

           0       0.98      0.92      0.95    240854
           1       0.91      0.97      0.94    212350

    accuracy                           0.94    453204
   macro avg       0.94      0.95      0.94    453204
weighted avg       0.95      0.94      0.94    453204

Confusion matrix (train set):

[[220937   5665]
 [ 19917 206685]]


For test set

              precision    recall  f1-score   support

           0       0.98      0.92      0.95     60243
           1       0.91      0.97      0.94     53059

    accuracy                           0.94    113302
   macro avg       0.94      0.95      0.94    113302
weighted avg       0.95      0.94      0.94    113302

Confusion matrix (test set):

[[55245  1406]
 [ 4998 51653]]


From the first row of the confusion matrix, we can see that the model estimates 55214 true valid transactions (True Negative) and 1437 false fraud detections (False Negative), while the second row shows that the classifier predicted 5094 as valid transaction even though in reality they are fraudulent (False Positive) and 51557 true fraudulent transactions (True Positive).

In [29]:
model.coef_

array([[ 0.9994759 , -1.25695853, -1.01020121,  0.10713968,  0.55877589,
        -0.97370506, -0.1958156 , -0.64624502, -0.74379332, -0.38558595,
         0.26101054,  0.28374334,  0.17416607, -0.25600728,  0.41617565,
         0.45191171, -0.1481852 ,  0.19306121, -0.61734848,  0.12198942,
        -0.10275597, -0.14145935, -0.0556177 , -0.41089475, -0.54252187]])

In [30]:
scores = cross_val_score(model, X, y, cv=5)
print('The accuracy score evaluation metric of logistic regression is :{}'.format(np.mean(scores)))

The accuracy score evaluation metric of logistic regression is :0.9414004439475839


### XGBoost Classifier

In [31]:
print('Model : XGBoost Classifier:\n\n')
model = XGBClassifier()
model.fit(X_train, y_train)
print('For train set\n')
print(classification_report(model.predict(X_train), y_train))
print('Confusion matrix (train set):\n')
print(confusion_matrix(y_train, model.predict(X_train)))

print()
print()

print('For test set\n')
print(classification_report(model.predict(X_test), y_test))
print('Confusion matrix (test set):\n')
print(confusion_matrix(y_test, model.predict(X_test)))

Model : XGBoost Classifier:


[11:36:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
For train set

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    226571
           1       1.00      1.00      1.00    226633

    accuracy                           1.00    453204
   macro avg       1.00      1.00      1.00    453204
weighted avg       1.00      1.00      1.00    453204

Confusion matrix (train set):

[[226571     31]
 [     0 226602]]


For test set

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56607
           1       1.00      1.00      1.00     56695

    accuracy                           1.00    113302
   macro avg      

From the first row of the confusion matrix, we can see that the model predicted 56607 True Negative and 44 False Negative, while the second row shows that the classifier predicted 56651 True Positive and 0 False Positive.

In [32]:
scores = cross_val_score(model, X, y, cv=5)
print('The accuracy score evaluation metric of the XGBoost Classifier is :{}'.format(np.mean(scores)))

[11:39:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:41:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:43:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:44:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

# Conclusions

In the beginning, we started working on a dataset composed by 284807 observations in which only the 0.172% of them are classified as fraudulent, making them really imbalanced data. \
We start cleaning the data, checking for missing data, duplicated data and outliers. We found no missing values and 1081 duplicates, which we proceeded to delete. With regard to the outliers, in order to avoid introducing bias that could lead us to erroneus conclusions, we decided to not delete them. Instead, we changed the outliers values: if it exceeded the upper bound, we would set it at the upper bound value and viceversa. \
At the end of this process, we got 283726 observations left, 0.167% of which are frauds.\
\
We then proceeded to select the most important features for prediction through the computation of the Information Value, which shows the relevance of almost all the considered variables. As we got really good results from the computation of the IV, we checked for the presence of high correlations between the given independent variables and the dependent one. From this last passage, we found that the variable "Amount" seems to be strongly correlated to the class and, because of this, we get rid of it in the continuation of the analysis. \
\
In the last part of our work, we evaluated and compared 2 predictive models: the Logistic Regression and the XGBoost Classifier. \
For each of them, we computed the confusion matrix and the accuracy score to measure the accuracy of our estimations and the F1-score to measure the recall and the precision of the prediction. All of these tools showed that both of the models considered seem to be good predictors of the credit card fraud, even though the XGBoost Classifier is more accurate, with an accuracy score of 0.9991. \
\
In conclusion we can say that the variables considered in this analysis are good for the prediction of credit card fraud, which can be done accurately through the Logistic Regression or the XGBoost Classifier, even though the second one seems more accurate according to both the cross-validation score and the f1-score.